# Agenda, week 5

1. Q&A
2. What are modules? What do they give us?
3. The various forms of the `import` statement
4. Developing a simple module
5. What happens when we `import` a module -- the special `__name__` variable
6. Python's standard library
7. PyPI and `pip` -- installing third-party modules
8. A little bit about `uv`
9. Using third-party modules -- how do you find, choose, download, and use them (and do that safely)?
10. AMA -- ask me anything
11. What next? Now that you've finished this course, where do you go for additional Python learning, resources, etc?

# What are modules? What do they give us?

We've talked a number of times about the "Don't repeat yourself" rule in programming, aka "DRY." If you have written something once, you shouldn't repeat it!

1. If the same line is repeated several times in a row, we should replace that with a loop.
2. If the same code is repeated several times in a program, we should replace that with a function.
3. If the same code is repeated several times across multiple programs, we can use a *library*.

A library is a term used in many programming languages to describe either code or data that was defined by someone else, in the past, which you can load into ("borrow," if you will) your program. You don't have to reinvent the wheel this way!

Just about every programming language supports libraries. This way, you can save yourself time in the future, or you can save your colleagues time, or they can save you time.

Moreover, if there is a bug in a library, it can be fixed once and affect all of our programs. Or if there's an optimization/fix/improvement, then we can fix it once, and affect all of our programs.

In Python, we call our libraries "modules" and "packages":

- A module is a single file in which we've defined Python functions and/or variables
- A package is a directory containing one or more modules.

When we use a module in Python, we get to reuse code and data that other people have defined.

A module also defines what we call a "namespace," a way to ensure that variable names are separated from one another, so different pieces of your program don't step on each other's variables.

# AP: What do you mean by "separate namespaces" for data and functions?

If I have a software product called ABCDE, and I take out a trademark on that product, then no one else in the software industry can have a product called ABCDE, because that would be confusing.

But if someone comes out with a car and calls it ABCDE, then I cannot stop them from having that trademark, because it's a different field.

You could say that even though the name is the same, it's in two different "namespaces," it has two different domains.

Another example: My name is "Reuven," and there are other Reuvens around. How do we distinguish between me and the others? We use last names! Last names serve as a "namespace," ensuring that I am easily distinguished from other Reuvens out there.

The problem of namespacing is basically that we need to resolve the ambiguity when there are multiple thigns with the same name.

In many languages, the namespace for functions and the namespace for variables is kept separate. You can have variables `a`, `b`, and `c`, and these names have nothing whatsoever to do with the functions called `a`, `b`, and `c`. They will never interfere with one another. (Whether this is a good idea for you to do in your program is separate.)

Python doesn't make this distinction. In Python, we have a single namespace, a single last name, a single domain for trademarks, for all of the things we define, both functions and variables. 

- If you define a variable `print`, you have just erased (in some ways) the builtin function `print`.   Only one can exist at a time! (This isn't quite true in the case of `print` and other builtin functions, but it will feel that way if you make such a definition!)
- If you define a function `total` that takes a bunch of numbers and returns their total, and then you invoke the function and assign the result to a variable named `total`, you have now erased the function's definition! Because now `total` is an integer, not a function.

Among other things, this means that you have to be a bit careful about what names you give to variables and functions, to ensure that they don't collide in this sort of way.

When we use `def` to define a function, we're really assigning to a variable. 

# How do we use a module in Python?

Python comes with a large collection of modules known as the "standard library." One of the modules in there is `random`, which contains a large number of functions having to do with random values -- getting random integers, choosing random elements of a list, etc.

If we want to use that functionality (rather than invent it ourselves), we need to tell Python that we want to load up the `random` module. How do we do that?

We use the `import` statement. It looks like this:

In [1]:
import random

# Dive into the syntax of `import`

1. We say `import`. Note that it's not a function! We don't put the name to its right in `()`.
2. The name to its right, unlike *most* programming languages I've used in the last, is not a string indicating the filename that we want to import or read. Rather, it's the name of the variable into which the module's values will be assigned.
3. There is no (easy, standard) way to use `import` with the name of a file. Rather, you provide the name of the module, and Python uses that as the basis for finding and loading the module you named.

Now that I've imported `random`, what can I do with it? 

Answer: Use any/all of the values that it now contains.

Well, what does it contain?

We can find out in a few different ways. The easiest (in Jupyter) is to run the `dir` function, which returns a list of strings, names that can go after a `.` after the module name.

If `dir(random)` includes the string `randint`, then we can use `random.randint` as a value or function, depending on what it is. I know that `random.randint` is a function, and we can then call

    random.randint(0, 100)

to get a random integer from 0-100.

If you want documentation showing not just the names, but what they do, then you can go to `docs.python.org` and look up the `random` module.

# Exercise: Guessing game

1. Have the computer choose a random integer from 0-100 using `random.randint` and assign to `number`.
2. Ask the user, repeatedly, to guess a number:
    - If it's too high, say "too high"
    - If it's too low, say "too low"
    - Otherwise, say "just right" and exit
3. If it's not right, then keep going, and ask the user to guess again.
4. When exiting the program, print not only the number, but how many guesses it took.

In [3]:
import random

number = random.randint(0, 100)
guess_counter = 0

while True:
    s = input('Guess: ').strip()

    if not s.isdigit():
        print('Not numeric; try again!')
        continue

    guess = int(s)
    guess_counter += 1

    if guess == number:
        print('You got it!')
        break
    elif guess < number:
        print('Too low!')
    else:
        print('Too high!')

print(f'Number was {number}, got it in {guess_counter} guesses.')        

Guess:  50


Too low!


Guess:  75


Too high!


Guess:  62


...

Too high!


Guess:  52


Too high!


Guess:  51


You got it!
Number was 51, got it in 7 guesses.
